<a href="https://colab.research.google.com/github/YoushaaMurhij/HackerEarth_Challenges/blob/master/Detect_emotions_of_your_favorite_toons.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# HackerEarth Deep Learning challenge: Detect emotions of your favorite toons

Downloading the dataset:

In [0]:
!wget https://he-s3.s3.amazonaws.com/media/hackathon/hackerearth-deep-learning-challenge-emotion-detection-tom-jerry-cartoon/detect-emotions-of-your-favorite-toons-7d2c0f23/96714c94-6-Dataset.zip testtt

In [0]:
!unzip -q Dataset.zip

In [0]:
import cv2
import math
import matplotlib.pyplot as plt
import pandas as pd
from keras.preprocessing import image
import numpy as np
from skimage.transform import resize

In [0]:
%matplotlib inline

Generating Training dataset from video:

In [7]:
count = 0
videoFile = "Train Tom and jerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="frame%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


Generating Testing dataset from video:

In [12]:
count = 0
videoFile = "Test Tom and Jerry.mp4"
cap = cv2.VideoCapture(videoFile)
frameRate = cap.get(5) #frame rate
x=1
while(cap.isOpened()):
    frameId = cap.get(1) #current frame number
    ret, frame = cap.read()
    if (ret != True):
        break
    if (frameId % math.floor(frameRate) == 0):
        filename ="test%d.jpg" % count;count+=1
        cv2.imwrite(filename, frame)
cap.release()
print ("Done!")

Done!


In [71]:
data.Emotion.unique()

array(['happy', 'surprised', 'angry', 'Unknown', 'sad'], dtype=object)

In [0]:
tg_dict = {"happy":0, "surprised": 1, "angry": 2,"Unknown":3,"sad":4}
def label_encode(x):
    return tg_dict[x]

In [0]:
data = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')
X = []
for img_name in data.Frame_ID:
    img = plt.imread('' + img_name)
    X.append(img)
X = np.array(X)

X_test = []
for img_name in test.Frame_ID:
    img = plt.imread('' + img_name)
    X_test.append(img)
X_test = np.array(X_test)

from keras.utils import np_utils
#train_y = np_utils.to_categorical(data.Emotion)
#train_y = pd.Categorical(data.Emotion)
train_yy = pd.Categorical(data.Emotion, categories=data.Emotion.unique()).codes
train_y = np_utils.to_categorical(train_yy)

#test_y = np_utils.to_categorical(test.Emotion)
image = []
for i in range(0,X.shape[0]):
    a = resize(X[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    image.append(a)
X = np.array(image)

iimage = []
for i in range(0,X_test.shape[0]):
    a = resize(X_test[i], preserve_range=True, output_shape=(224,224,3)).astype(int)
    iimage.append(a)
X_test = np.array(iimage)

In [97]:
train_y

array([[1., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]], dtype=float32)

In [0]:
from keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout

In [99]:
X_test.shape

(186, 224, 224, 3)

In [0]:
#X_test = X_test.reshape(186, 7*7*512)


In [100]:
X_test.shape

(186, 224, 224, 3)

In [101]:
X.shape

(298, 224, 224, 3)

In [0]:
X = preprocess_input(X, mode='tf')
X_test = preprocess_input(X_test, mode='tf')

X_train, X_valid, y_train, y_valid = train_test_split(X, train_y, test_size=0.3, random_state=42)

base_model = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
X_train = base_model.predict(X_train)
X_valid = base_model.predict(X_valid)
X_test  = base_model.predict(X_test)

X_train = X_train.reshape(208, 7*7*512)
X_valid = X_valid.reshape(90, 7*7*512)
X_test  = X_test.reshape(186, 7*7*512)

train   = X_train/X_train.max()
X_valid = X_valid/X_train.max()
X_test  = X_test/X_test.max()


In [0]:
model = Sequential()
model.add(InputLayer((7*7*512,)))    # input layer
model.add(Dense(units=1024, activation='sigmoid'))   # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=512, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(units=256, activation='sigmoid'))    # hidden layer
model.add(Dropout(0.5))      # adding dropout
model.add(Dense(5, activation='softmax'))            # output layer

In [0]:

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
class_weights = compute_class_weight('balanced',np.unique(data.Emotion), data.Emotion)  # computing weights of different classes

from keras.callbacks import ModelCheckpoint
filepath="weights.best.hdf5"

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]      # model check pointing based on validation loss

model.fit(train, y_train, epochs=500, validation_data=(X_valid, y_valid), class_weight=class_weights, callbacks=callbacks_list)

model.load_weights("weights.best.hdf5")
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
scores = model.evaluate(X_valid, y_valid)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))

Train on 208 samples, validate on 90 samples
Epoch 1/500
208/208 [==============================] - 1s 7ms/step - loss: 0.6341 - acc: 0.7933 - val_loss: 1.1373 - val_acc: 0.6000

Epoch 00001: val_loss improved from inf to 1.13733, saving model to weights.best.hdf5
Epoch 2/500
208/208 [==============================] - 0s 919us/step - loss: 0.5433 - acc: 0.8269 - val_loss: 1.0657 - val_acc: 0.6556

Epoch 00002: val_loss improved from 1.13733 to 1.06573, saving model to weights.best.hdf5
Epoch 3/500
208/208 [==============================] - 0s 889us/step - loss: 0.5011 - acc: 0.8221 - val_loss: 1.1707 - val_acc: 0.6111

Epoch 00003: val_loss did not improve from 1.06573
Epoch 4/500
208/208 [==============================] - 0s 842us/step - loss: 0.4005 - acc: 0.8798 - val_loss: 1.1067 - val_acc: 0.6667

Epoch 00004: val_loss did not improve from 1.06573
Epoch 5/500
208/208 [==============================] - 0s 866us/step - loss: 0.3199 - acc: 0.9135 - val_loss: 1.1971 - val_acc: 0.6333


In [0]:
preds = model.predict(X_test)

In [0]:
predictions=[]
for i in preds:
    predictions.append(np.argmax(i))

In [0]:
test['Emotion'] = predictions

In [0]:
gt_dict = dict((v,k) for k,v in tg_dict.items())

def inverse_encode(x):
    return gt_dict[x]

test['Emotion'] = test['Emotion'].apply(inverse_encode)

In [110]:
test.head(4)

,Frame_ID,Emotion
0,test0.jpg,happy
1,test1.jpg,happy
2,test2.jpg,Unknown
3,test3.jpg,Unknown


In [0]:
test.to_csv('Submission.csv',header=True,index = None)